In [31]:
import re

# Retira todas tags de um texto HTML
def cleanHtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

# Retira os parênteses de um nome
def cleanName(raw_name):
  cleanr = re.compile('\(.*?\)')
  cleantext = re.sub(cleanr, '', raw_name)
  return cleantext

# Retira o conteúdo entre dois marcadores de um texto HTML
def extractContent(raw_html, mark1, mark2):
  contentr = re.search(mark1 + '.*?' + mark2, raw_html)[0]
  contentr = contentr.split(mark1)[1].rsplit(mark2)[0]
  return cleanHtml(contentr)

# Retira a apresentação do orador no início da íntegra - vai até o primeiro " -" depois do nome do autor
def extractOradorFromContent(content, orador):
    extracao = re.search(orador + '.*?$', content, flags=re.IGNORECASE)
    if extracao != None and extracao.start(0) < 200:
        contentr = extracao[0]
        extracao = re.search(' - .*?$', contentr, flags=re.IGNORECASE)
        if extracao != None and extracao.start(0) < 200: ### REAVALIAR. Não funciona para pronunciamento encaminhado pelo orador muito grande
            contentr = extracao[0]
            contentr = re.sub(' - ', '', contentr, flags=re.IGNORECASE)
        else:
            contentr = re.sub(orador + '[^a-zA-Z0-9]*', '', contentr, flags=re.IGNORECASE)
    else:
        contentr = content

    # Verifica se o pronunciamento foi encaminhado pelo autor
    encaminhado = re.match('PRONUNCIAMENTO ENCAMINHADO.*?$', content, flags=re.IGNORECASE)
    if encaminhado != None:
        # Verifica se ainda há marcação de encaminhado no início da íntegra - content"r"
        encaminhado = re.match('PRONUNCIAMENTO ENCAMINHADO.*?$', contentr, flags=re.IGNORECASE)
        if encaminhado != None:
            contentr = encaminhado[0]
            encaminhado = re.search('\) - .*?$', contentr, flags=re.IGNORECASE)
            if encaminhado != None:
                contentr = encaminhado[0]
                contentr = re.sub('\) - ', '', contentr, flags=re.IGNORECASE)
        contentr = 'PRONUNCIAMENTO ENCAMINHADO PELO ORADOR ' + contentr

    return contentr

# Retira a manifestação do presidente ao final da íntegra, se houver
def extractPresidenteFromContent(content):
    presidr = re.search('O SR\. PRESIDENTE\s*?\(.*?$', content, flags=re.IGNORECASE)
    if presidr != None:
        contentr = content.rsplit(presidr[0])[0].rstrip()
        
        # Verifica se há pronunciamento encaminhado pelo autor concatenado ao final do discurso
        encaminhado = re.search('PRONUNCIAMENTO ENCAMINHADO PELO ORADOR.*?$', presidr[0], flags=re.IGNORECASE)
        if encaminhado != None:
            contentr = contentr + ' ' + encaminhado[0]
        
    else:
        contentr = content
    return contentr

In [33]:
#print(extractPresidenteFromContent(extractOradorFromContent("O SR. EDMILSON RODRIGUES    (PSOL-PA. Sem revis\u00e3o do orador.) - Sr. Presidente, eu j\u00e1 denunciei o crime do Presidente Temer, que em breve ter\u00e1 o mandato suspenso. E tenho f\u00e9 no povo, na indigna\u00e7\u00e3o do nosso povo, que vai come\u00e7ar a se rebelar e a ir para as ruas. Muitos Deputados que votaram contra a investiga\u00e7\u00e3o j\u00e1 est\u00e3o sentindo o drama de apoiar um governo t\u00e3o antipovo, t\u00e3o destruidor dos direitos da popula\u00e7\u00e3o.    Os cortes na \u00e1rea de educa\u00e7\u00e3o foram brutais. A partir de setembro, as bolsas de inicia\u00e7\u00e3o cient\u00edfica, mestrado, doutorado e p\u00f3s-doutorado que o CNPq financia para mais de 100 mil cientistas brasileiros n\u00e3o estar\u00e3o mais garantidas. Isso \u00e9 um crime contra a soberania nacional, contra o desenvolvimento cient\u00edfico, tecnol\u00f3gico e social do Pa\u00eds.    Abaixo a viol\u00eancia! Abaixo Temer!      O SR. PRESIDENTE    (Carlos Manato) - Obrigado, Deputado.       PRONUNCIAMENTO ENCAMINHADO PELO ORADOR    Sr. Presidente, Sras. e Srs. Deputados, venho a esta tribuna hoje para falar mais uma vez sobre o corte de recursos empreendido pelo Governo Federal. O foco dos cortes sempre est\u00e1 em \u00e1reas estrat\u00e9gicas para o desenvolvimento social e tecnol\u00f3gico desta Na\u00e7\u00e3o e mira sempre nossos direitos fundamentais.    Refiro-me hoje, com grande preocupa\u00e7\u00e3o, ao contingenciamento de recursos da Educa\u00e7\u00e3o, Pasta que sofreu o maior corte e que, hoje, s\u00f3 tem verba para manter as universidades funcionando at\u00e9 setembro. Caso n\u00e3o se descontingenciem os recursos, essas institui\u00e7\u00f5es v\u00e3o parar. N\u00e3o haver\u00e1 verba para pagar sal\u00e1rios de professores e servidores, bolsas de pesquisa nem mesmo \u00e1gua e luz. \u00c9 uma situa\u00e7\u00e3o alarmante, urgente e, sobretudo, vergonhosa.    Na \u00faltima semana, o Presidente do CNPq, a principal ag\u00eancia de fomento \u00e0 pesquisa cient\u00edfica e tecnol\u00f3gica do Pa\u00eds, M\u00e1rio Borges, anunciou que as bolsas de inicia\u00e7\u00e3o cient\u00edfica, mestrado, doutorado e p\u00f3s-doutorado - que atingem, portanto, cerca de 100 mil pesquisadores - s\u00f3 poderiam ser pagas at\u00e9 setembro. Na pr\u00e1tica, a pesquisa no Brasil sente o corte anunciado no in\u00edcio do ano de 44% nos valores previstos para 2017 do Minist\u00e9rio da Ci\u00eancia, Tecnologia, Inova\u00e7\u00f5es e Comunica\u00e7\u00f5es - MCTIC, o que se traduziu em uma redu\u00e7\u00e3o de 572 milh\u00f5es de reais nos repasses de 1,3 bilh\u00e3o de reais esperados pelo CNPq. O Presidente do CNPq deixou clara a situa\u00e7\u00e3o: a pior que o \u00f3rg\u00e3o vive em 66 anos.    Diante do an\u00fancio, o Ministro da Ci\u00eancia, Tecnologia, Inova\u00e7\u00f5es e Comunica\u00e7\u00f5es, Gilberto Kassab, recebeu o Presidente e disse estar otimista quanto ao descontingenciamento dos recursos. Ele se refere a uma poss\u00edvel negocia\u00e7\u00e3o com as Pastas da Fazenda e do Planejamento. Contudo, S.Exa. deixou claro que as contas s\u00f3 est\u00e3o garantidas at\u00e9 agosto.    O descontingenciamento, contudo, tamb\u00e9m deve ocorrer para outra Pasta: a da Educa\u00e7\u00e3o. Os cortes anunciados em mar\u00e7o atingiram 3,6 bilh\u00f5es de reais de despesas diretas do Minist\u00e9rio da Educa\u00e7\u00e3o, al\u00e9m de 700 milh\u00f5es de reais em emendas parlamentares para a \u00e1rea da educa\u00e7\u00e3o. Levando em conta o total previsto, a Pasta realizou um contingenciamento de 15% do or\u00e7amento para o custeio das universidades e 40% da verba para obras. Desde ent\u00e3o, as universidades t\u00eam feito de tudo para se adequar ao novo or\u00e7amento: demiss\u00e3o de terceirizados, obras paradas, contas atrasadas. Mas dezenas j\u00e1 anunciam: se n\u00e3o liberarem mais recursos, elas podem parar a partir de setembro.    \u00c9 uma vergonha que uma Na\u00e7\u00e3o como o Brasil, cujo PIB chega a mais de 1,5 trilh\u00e3o de reais, esteja prestes a paralisar a pesquisa nacional e fechar portas de universidades por falta de recursos. Essa vergonha \u00e9 ainda maior quando j\u00e1 \u00e9 de conhecimento em todo o mundo que, para livrar seu nome de investiga\u00e7\u00e3o, Temer tenha aliviado uma d\u00edvida de 10 bilh\u00f5es de reais com a bancada ruralista.    N\u00e3o se trata apenas de uma crise econ\u00f4mica. Ela \u00e9, sobretudo, moral. O Pa\u00eds assiste estarrecido o vergonhoso espet\u00e1culo da corrup\u00e7\u00e3o nos n\u00edveis mais elevados da institucionalidade nacional a proteger as contas daqueles milion\u00e1rios que sempre lucraram \u00e0 custa do povo trabalhador. Escolhe-se paralisar a pesquisa e fechar as universidades como se essa preocupa\u00e7\u00e3o fosse menor do que agradar ao mercado financeiro.    Dirijo apelo aqui, portanto, ao Ministro da Fazenda, Henrique Meirelles, e ao Ministro do Planejamento, Dyogo Oliveira, para que tenham sensibilidade em compreender a gravidade do que est\u00e1 em jogo e que descontingenciem os recursos da Educa\u00e7\u00e3o e da Ci\u00eancia e Tecnologia. \u00c9 nosso desenvolvimento e futuro que est\u00e3o em quest\u00e3o.", "edmilson rodrigues")))
#print(extractPresidenteFromContent(extractOradorFromContent("A SRA. TATIANA ELIAS ROQUE   - Boa tarde a todas e a todos.    Gostaria de agradecer ao Deputado Celso Pansera por organizar este evento aqui no plenário.    Eu gostaria de começar com uma frase de Carlos Chagas Filho sobre a universidade:    Aqui se ensina porque se pesquisa   .    Acho que estamos falando aqui o tempo todo de um tripé, de três atividades indissociáveis. Uma delas é o desenvolvimento do País, o desenvolvimento sustentável, que passa or encontrar modos originais de investimento em inovação e maneiras de fazer com que esse desenvolvimento leve o Brasil para o futuro, com a economia do conhecimento, com áreas que hoje em dia não são tão estimuladas no nosso desenvolvimento industrial.    Em segundo lugar, a Ciência e a Tecnologia, que são fundamentais para esse desenvolvimento.    Em terceiro lugar, a base de sustentação desse tripé, a universidade pública. Não só porque a universidade pública vai fornecer esses cérebros e esses quadros para a Ciência e a Tecnologia, para que o nosso modelo de desenvolvimento se renove, mas também porque hoje a universidade pública é um laboratório de convivência entre as diferenças, devido às políticas de expansão e de democratização do acesso que foram feitas nos últimos anos. É fundamental que esses estudantes que acabaram de entrar na universidade, a quem nunca foi dada essa oportunidade, possam se manter na universidade. Isso só vai acontecer se essa universidade continuar sendo pública e continuar contando com investimentos do Governo, do poder público.    Eu acho que precisamos, urgentemente, manter os níveis de investimento público na universidade e em Ciência e Tecnologia, tanto para que possamos pensar um novo modelo de desenvolvimento com a manutenção da pesquisa, que é fundamental, quanto para que possamos manter um modelo de combate às desigualdades, que já funciona hoje dentro das nossas universidades públicas. Então, justamente por essa dupla via, precisamos manter desde bolsas, para que esses estudantes que entram pelas cotas possam se manter na universidade, com a infraestrutura funcionando, como o bandejão e o alojamento, sem os quais esses estudantes não poderão continuar frequentando a universidade, até o investimento nos laboratórios, na infraestrutura de pesquisa, nas bolsas de pós-graduação e nas verbas para projetos de pesquisa.    Por isso tudo, é fundamental o investimento público na universidade, em Ciência e Tecnologia.    A Associação dos Docentes da Universidade Federal do Rio de Janeiro - ADUFRJ, em conjunto com a Associação dos Docentes da Universidade de Brasília - ADUnB e com a Associação dos Professores da Universidade Federal de Minas Gerais, em parceria com a Sociedade Brasileira para o Progresso da Ciência - SBPC e com a Associação Nacional de Pós-Graduandos - ANPG, lançou a campanha   Conhecimento sem Cortes   , que tem por objetivo sensibilizar a sociedade para o fato de que o que é feito hoje na universidade e na área de Ciência e Tecnologia é fundamental e necessita da manutenção dos investimentos.    Uma das primeiras ações de nossa campanha foi a instalação do Tesourômetro, um painel digital, na Universidade Federal do Rio de Janeiro. Vão ser instalados outros na UFMG e em Brasília, em breve. Esse Tesourômetro contabiliza o que nós temos perdido, desde 2015, em investimentos na área. Reúne os investimentos nas universidades públicas federais, na CAPES e no Ministério da Ciência e Tecnologia. Pasmem: pelas nossas contas, nós estamos perdendo 500 mil reais por hora desde 2015.    Ou seja, a situação é muito grave. Eu gostaria muito que nós nos mantivéssemos mobilizados e ligados a essa campanha. Convido todas e todos a participarem da campanha   Conhecimento sem Cortes   , que já está nas redes sociais, na Internet etc.    Obrigada.   (Palmas.)        O SR. PRESIDENTE    (Deputado Celso Pansera) - Obrigado, Tatiana Roque, da ADUFRJ. O Tesourômetro é um critério bom. Sei que estão procurando um prédio no centro de Brasília para instalá-lo. Já me falaram...    O prédio da CNI seria bom, é verdade, tem um bom visual.", "Tatiana Roque")))
#print(extractOradorFromContent('O SR. HOMERO MATEUS   - Bom dia a todos os membros da Mesa e aos demais presentes.    Primeiramente, nossos agradecimentos ao Presidente Vitor Lippi, ao Deputado Celso Pansera, o grande patrono, digamos assim, desta nossa reunião de hoje.    Eu até havia escrito alguma coisa, mas resolvi mudar um pouco a minha estratégia de fala.    Hoje já dissemos aqui algumas dezenas de vezes que o setor de Ciência e Tecnologia, a parte de pesquisa e desenvolvimento, é estratégico para o País. Acho que não temos nem mais que discutir isso, porque é a maior verdade de todas que foram ditas aqui hoje.    O corte que aconteceu na parte dos recursos orçamentários, de 44%, além de prejudicar a pesquisa, prejudicou também e muito o setor produtivo, ou seja, as empresas, aquelas que transformam as pesquisas em produtos que vão trazer riqueza para o País. Talvez este seja o ponto mais preocupante para o nosso setor.    Quando um corte acontece, o que nós sentimos não é só a falta de dinheiro, é a falta da segurança jurídica, que compreende todos esses passos. Ou seja, o setor não pode apenas fazer investimento prevendo o que vai acontecer no ano seguinte, quando os seus próprios contratos com o Estado acabam sendo suspensos pela falta exatamente do orçamento.    A tecnologia foi a grande mola mestra propulsora de tudo o que temos hoje de riqueza em todos os países desenvolvidos, a indústria da inteligência. Hoje, no Brasil, falamos muito nas   startup   s, coisa que, em alguns lugares, já estão deixando de lado. Não conseguimos fazer com que as   startups   decolem no Brasil.    Como o Deputado bem colocou, nós estamos vivendo o momento das cidades inteligentes, dessas coisas todas, das pesquisas voltadas para IoT e não temos o mais básico de tudo: somos carentes de infraestrutura. Não conseguimos fazer essas coisas porque o básico não foi feito como dever de casa.    Quando cortamos dinheiro para pesquisa e desenvolvimento, além de não desenvolvermos   startups   , nós perdemos esse capital humano para o mundo. Aqui em Brasília nós temos uma ligação muito forte com a Universidade de Brasília e vemos que os alunos graduandos, ou de mestrado, de qualquer curso que seja, que resolvem empreender, veem que não conseguem e acabam sendo seduzidos por outras possibilidades no exterior. E assim vamos ficando cada vez mais à míngua.    Como o Presidente da FAP aqui de Brasília colocou, nós temos em Brasília, sim, um movimento interessante com relação a esse fomento de pesquisa para a área acadêmica, para as   startups   e, parece-me, no futuro também para as empresas, em conjunto com as universidades. Esse talvez seja o maior paradigma a se quebrar. Quando aparece um empresário para falar sobre corte de verba, só se pensa e se imagina que estamos reclamando porque nós os milionários da indústria de tecnologia deixamos de ganhar dinheiro. Não. Quem está deixando de ganhar dinheiro não são os empresários, é o nosso País, pois divisas poderiam estar sendo produzidas, riquezas poderiam estar sendo geradas.    Além de o setor de tecnologia do País ser pujante - e, graças a Deus, nós somos muito fortes -, o Governo é um grande consumidor de tecnologia. Quando o Governo fecha essa porta, nós estancamos todas as partes de um processo de produção.    Eu acredito de verdade que um dia ainda teremos, se Deus quiser, um Prêmio Nobel da Ciência para o Brasil. A única certeza que no momento eu posso ter, com esses cortes, é de que corremos um risco muito grande de esse cidadão brasileiro não morar no Brasil.    Obrigado, Sr. Presidente.   (Palmas.)' , "Homero Mateus"))
#print(extractOradorFromContent("O SR. PRESIDENTE    (Pedro Uczai) - Tem a palavra o Deputado Valmir Assunção, por até 1 minuto.      O SR. VALMIR ASSUNÇÃO    (PT-BA. Pela ordem. Sem revisão do orador.) - Sr. Presidente, Sras. e Srs. Deputados, eu sou daqueles que acreditam que Sérgio Moro hoje cumpriu mais um papel no golpe que foi dado na Constituição deste País.    Estou dizendo isso porque ele estabeleceu uma pena para o Presidente Lula sem ter nenhuma prova, por saber que hoje se iniciou o processo de discussão sobre o afastamento do Presidente Michel Temer. Então, ele quer tirar o foco desse processo contra Michel Temer e, para isso, condena o Presidente Lula.    E Sérgio Moro sabe que essa condenação não vai inviabilizar a eleição do Lula no ano que vem, porque, para não ser candidato, é preciso que Lula seja condenado por um colegiado, em segunda instância, e esse colegiado já disse que não concorda com a decisão de Moro.    Nesse aspecto, o objetivo dessa condenação é simplesmente tumultuar a situação. A Direita brasileira no dia de hoje está comemorando, mas podem ter certeza de que em 2018 nós vamos ganhar essa eleição, para retomar o crescimento e o fortalecimento do nosso Brasil.    Deixo registrada a nossa solidariedade ao Presidente Lula.","Valmir Assunção"))
print(extractPresidenteFromContent(extractOradorFromContent("O SR. PRESIDENTE    (Deputado Celso Pansera) - Para encerrar, tem a palavra o Sr. Sávio Raeder, representante do Ministro Gilberto Kassab.    O Sávio é carioca e também botafoguense.      O SR. SÁVIO TÚLIO OSELIERI RAEDER   - Já sofri mais. Agora estou colhendo alguns risos.    Boa tarde a todos.    Primeiro, quero cumprimentar a Mesa: a Gianna Sagazio, da CNI - é uma alegria poder estar aqui neste momento de luta pela Ciência; o Prof. Luiz Davidovich, que muito nos honra com as defesas muito firmes, muito sólidas do nosso sistema de Ciência e Tecnologia; o Deputado e ex-Ministro Celso Pansera, um grande aliado em todas as nossas lutas; e a Profa. Helena Nader, a quem estendo um caloroso abraço pelos 10 anos de luta à frente da SBPC - é com muito orgulho que contamos esse tempo todo com a sua brava e denodada luta à frente do nosso sistema.    O Parlamento é, sem dúvida, uma Casa que vem defendendo com força o nosso setor de Ciência, Tecnologia e inovação. Em 2015, com a Emenda Constitucional nº 85, tivemos ganhos bastante relevantes, como a institucionalização do Sistema Nacional de Ciência e Tecnologia, ainda pendente de institucionalização por meio de lei.    Em 2016, no ano seguinte, obtivemos a regulamentação da emenda constitucional, como marco legal, com a Lei nº 13.243. Infelizmente, houve oito vetos, ainda pendentes de reversão. Existem algumas iniciativas no Congresso para isso. Destacaria o Projeto de Lei do Senado nº 226, de 2016, de autoria do Senador Jorge Viana e com a relatoria do Senador Cristovam Buarque. Se tudo der certo, conseguiremos como vitória deste ano a reversão desses vetos.    A criação do marco legal é, sem dúvida, um avanço fantástico. Lutamos duramente pela sua regulamentação. Existem mais dificuldades internas do que externas para o alcance do que foi colocado, em termos de Constituição, em termos de marco legal. Mas seguimos. Seguimos também em outras frentes. Por exemplo, a retomada do Conselho Nacional de Ciência e Tecnologia, o CCT. Estamos cumprindo uma agenda consecutiva de debates, para a tomada de ações efetivas com base nesse fórum, que é o fórum máximo do nosso setor, presidido pelo Presidente da República.    Quero agradecer ao representante da Fundação de Apoio à Pesquisa do Distrito Federal - FAP-DF por ter tratado da questão da Estratégia Nacional de Ciência, Tecnologia e Inovação - ENCTI. Ela foi debatida e aprovada no âmbito do CCT e tem um horizonte temporal de 2016 a 2022. Contamos com 12 áreas estratégicas e, neste momento, estamos trabalhando no desdobramento delas, em planos operacionais, em planos que vão detalhar de que modo vamos conseguir fazer, até 2022, as ações mais específicas e mais concretas para o nosso sistema.    Sem dúvida, as restrições orçamentárias nos apresentam grandes desafios, os cortes são realmente drásticos, mas seguimos fazendo todo o esforço possível para planejar o nosso sistema com base nessa ENCTI, que tem o horizonte temporal estendido para 2022.    Outra iniciativa, também de planejamento, da qual estamos nos aproximando, que tem horizonte temporal mais estendido, são os 17 Objetivos de Desenvolvimento Sustentável da ONU, que vão vigorar de 2016 a 2030, uma iniciativa à qual o mundo inteiro se associa e da qual o Ministério da Ciência e Tecnologia tem promovido grandes esforços para se aproximar.    No campo da regulação, o Jorge Mario Campagnolo, que está aqui, o Diretor da SETEC - Secretaria de Desenvolvimento Tecnológico e Inovação e Secretário Substituto, está à frente das discussões sobre a revisão da Lei do Bem. Além disso, temos a própria regulamentação da Lei 13.243 e estamos trabalhando firmemente também na revisão da Lei de Informática.    Vale destacar que esta semana tivemos um avanço relacionado às Fundações de Apoio, com a extensão do prazo de credenciamento de 2 anos para 5 anos. Esse era um gargalo muito grande no sistema. Esperamos tornar isso lei a partir da Medida Provisória nº 785.    Aqui nesta Casa há outra iniciativa muito relevante, que é a da criação dos fundos patrimoniais, um instrumento de alta relevância no exterior e que esperamos conseguir institucionalizar no País. Então, a criação dos fundos patrimoniais é uma ação também bastante relevante.    Em relação às ações mais concretas, em termos de editais, em termos de ações efetivas, o Marcelo Morales, do CNPq, representante do Mario Neto, já abordou várias delas. Sem dúvida, conseguimos ainda tocar algumas ações, apesar dos cortes, com muito esforço, com muita reorientação de iniciativas.    Eu destacaria também que vamos manter a Semana Nacional de Ciência e Tecnologia, a nossa principal ação de popularização, no Sistema Nacional de Ciência, Tecnologia e Inovação. É um grande evento, de dimensão nacional, realizado no mês de outubro.    Enfim, tratando mais especificamente dos cortes orçamentários, o desafio é enorme. Ontem pela manhã, no Senado Federal, foi colocado pelo nosso Secretário Executivo Elton Zacarias que esforços vêm sendo empreendidos pelo Ministro Gilberto Kassab junto ao Planejamento, junto à Fazenda, para recompor isso. Não é fácil a discussão, mas a expectativa é de que até setembro consigamos ter boas notícias nesse sentido. Tudo depende, obviamente, de novas receitas, de um novo respiro da economia, para conseguirmos fazer essa recomposição, na medida do possível.    O corte é drástico. Assim como concordo com o Secretário Elton, tenho que concordar com todas as falas feitas aqui, no mérito. Todas elas são realmente relevantes e direcionadas ao ponto correto da questão das dificuldades orçamentárias. Se não conseguirmos superar isso, se não conseguirmos superar a capacidade de financiamento, de investimento em Ciência, Tecnologia e inovação no País, sem dúvida alguma, não teremos nenhum futuro digno a apresentar aos nossos filhos, aos nossos netos; sem dúvida alguma, todo o nosso desenvolvimento econômico, social, ambiental vai estar comprometido por algumas gerações, por todos os motivos que já foram enumerados aqui.    Deputado Celso Pansera, mais uma vez agradeço a oportunidade de estar aqui. Parabenizo-o pela iniciativa. Sigamos em frente nesta luta.    Obrigado.   (Palmas.)        O SR. PRESIDENTE    (Deputado Celso Pansera) -       Obrigado, Sávio, representante do Ministro Gilberto Kassab.    Registro a presença de Jorge Mario Campagnolo, Secretário Substituto da Secretaria de Desenvolvimento Tecnológico e Inovação do Ministério da Ciência, Tecnologia, Inovações e Comunicações, que passou o dia de ontem conosco.    Obrigado pela presença.","Sávio Túlio Oselieri Raeder")))

Já sofri mais. Agora estou colhendo alguns risos.    Boa tarde a todos.    Primeiro, quero cumprimentar a Mesa: a Gianna Sagazio, da CNIé uma alegria poder estar aqui neste momento de luta pela Ciência; o Prof. Luiz Davidovich, que muito nos honra com as defesas muito firmes, muito sólidas do nosso sistema de Ciência e Tecnologia; o Deputado e ex-Ministro Celso Pansera, um grande aliado em todas as nossas lutas; e a Profa. Helena Nader, a quem estendo um caloroso abraço pelos 10 anos de luta à frente da SBPCé com muito orgulho que contamos esse tempo todo com a sua brava e denodada luta à frente do nosso sistema.    O Parlamento é, sem dúvida, uma Casa que vem defendendo com força o nosso setor de Ciência, Tecnologia e inovação. Em 2015, com a Emenda Constitucional nº 85, tivemos ganhos bastante relevantes, como a institucionalização do Sistema Nacional de Ciência e Tecnologia, ainda pendente de institucionalização por meio de lei.    Em 2016, no ano seguinte, obtivemos a regulamentaçã

In [288]:
extractContent("lstrip(mark1)etapa=5&numSessao=337.3.55.O&numQuarto=6&numOrador=2&numInsercao=0", "numInsercao=", "$") 

'0'

In [35]:
import requests, json, re

from datetime import datetime

# Obtém lista de discursos no Sitaq em um intervalo de tempo
def getDiscursosSitaqFast(initialDateTime, finalDateTime):
    p = {'siglaAplicacao':'novoDiscurso', 'ordenacao':'data', 'ordenacaoDir':'asc', 'dataInicial': initialDateTime, 'dataFinal': finalDateTime}
    response = requests.get("http://vvmchjar01.redecamara.camara.gov.br:8082/fastsearchWS/webservice/getConsulta", params=p)
  
    qtdPorPagina = json.loads(response.content)["qtdPorPagina"]
    qtdDePaginas = json.loads(response.content)["qtdDePaginas"]

    listaDiscursos = []

    for pagina in range(1, qtdDePaginas+1):
        
        p = {'siglaAplicacao':'novoDiscurso', 'ordenacao':'data', 'ordenacaoDir':'asc', 'dataInicial': initialDateTime, 'dataFinal': finalDateTime, 'pagina':pagina}
        response = requests.get("http://vvmchjar01.redecamara.camara.gov.br:8082/fastsearchWS/webservice/getConsulta", params=p)
        listaDiscursosHTML = json.loads(response.content)["indexProfile"]

        for discursoHTML in listaDiscursosHTML:
            
            # Verifica se o discurso tem íntegra e se não foi registrado em sessão solene
            # Nas sessões solenes, o Detaq coloca a ata da reunião onde deveria estar a íntegra do discurso do deputado
            if (discursoHTML["companyteaser"] != None) and (re.search("Não Deliberativa Solene", discursoHTML["tipoconteudo"]) == None):

                # Captura dados do autor do discurso
                autor = {}
                if discursoHTML['tipoautor'] != None: 
                    autor['tipoautor'] = discursoHTML['tipoautor'].lstrip().rstrip()
                if discursoHTML['autor'] != None: 
                    autor['nome'] = cleanName(discursoHTML['autor']).lstrip().rstrip().title()
                    if autor['tipoautor'] != None and autor['tipoautor'] == 'Deputado':
                        autor['id'] = autor['tipoautor'] + ' ' + autor['nome']
                    else:
                        autor['id'] = autor['nome']
                if discursoHTML['estado'] != None: 
                    autor['UF'] = discursoHTML['estado'].lstrip().rstrip()
                if discursoHTML['partido'] != None: 
                    autor['partido'] = discursoHTML['partido'].lstrip().rstrip()
                if discursoHTML['emails'] != None: 
                    autor['sexo'] = discursoHTML['emails'].lstrip().rstrip()
                    
                # Associa autor ao discurso
                discurso = {}                         
                discurso['autor'] = autor
                
                # ID do discurso formado por nu_sessao (generic3) + nu_quarto (generic4) + nu_sequencia (generic5)
                discurso['id'] = (extractContent(discursoHTML['url'], "numSessao=", "&") + " "
                                  + extractContent(discursoHTML['url'], "numQuarto=", "&") + " "
                                  + extractContent(discursoHTML['url'], "numInsercao=", '$') + " "
                                  + extractContent(discursoHTML['url'], "numOrador=", "&"))

                # Extrai íntegra do discurso do campo "companyteaser", no <body>
                discurso['integra'] = extractContent(discursoHTML["companyteaser"], "<body>", "</body>").lstrip().rstrip()
                discurso['original'] = discurso['integra']
                discurso['integra'] = extractOradorFromContent(discurso['integra'], autor['nome'])
                discurso['integra'] = extractPresidenteFromContent(discurso['integra'])
                
                # Indexação manual de termos feita pelo Departamento de Taquigrafia
                if discursoHTML['generic14'] != None: 
                    discurso['indexacao'] = discursoHTML['generic14'].lstrip().rstrip()

                # Data e hora do discurso
                if discursoHTML['docdatetime'] != None: 
                    #discurso['dtDiscurso'] = datetime.strptime(discursoHTML['docdatetime'].lstrip().rstrip(), '%Y-%m-%dT%H:%M:%SZ')
                    discurso['dtDiscurso'] = str(datetime.strptime(discursoHTML['docdatetime'].lstrip().rstrip(), '%Y-%m-%dT%H:%M:%SZ'))
                
                # Data e hora da última atualização do discurso
                if discursoHTML['processingtime'] != None: 
                    #discurso['dtAtualizacao'] = datetime.strptime(discursoHTML['processingtime'].lstrip().rstrip(), '%Y-%m-%dT%H:%M:%SZ')
                    discurso['dtAtualizacao'] = str(datetime.strptime(discursoHTML['processingtime'].lstrip().rstrip(), '%Y-%m-%dT%H:%M:%SZ'))

                listaDiscursos.append(discurso)

#            else:
#                print(discursoHTML["tipoconteudo"])

    return listaDiscursos

# Grava uma lista de discursos do Sitaq no Babel via API
def insertDiscursosSitaqBabel(listaDiscursos):

    for discurso in listaDiscursos:
    
        p = {'name':discurso['autor']['nome'],'author_type':discurso['autor']['tipoautor']}
        response = requests.get('http://babel.labhackercd.leg.br/api/v1/authors', params=p)
    
    'name': 'Erivânio Vasconcelos',
    'author_type': 'Cidadão',
    'gender': 'masculino',
    'birthdate': '1992-02-01',
    'cep': '72000000',
    


#discursos = getDiscursosSitaqFast('08/07/2017', '08/08/2017')

#print(discursos)

#arq = open('discursos.json', 'w')
#arq.write(json.dumps(discursos))
#arq.close()



In [133]:
import requests, json

# CEP base para representar os estados no cadastro dos deputados no Babel
CEP = {'AC':'69900', 'AL':'57000', 'AP':'68900', 'AM':'69000', 'BA':'40000', 'CE':'60000', 'DF':'70000', 'ES':'29000', 'GO':'72800', 'MA':'65000', 'MT':'78000', 'MS':'79000', 'MG':'30000', 'PA':'66000', 'PB':'58000', 'PR':'80000', 'PE':'50000', 'PI':'64000', 'RJ':'20000', 'RN':'59000', 'RS':'90000', 'RO':'78900', 'RR':'69300', 'SC':'88000', 'SP':'01000', 'SE':'49000', 'TO':'77000'}

# Channel ID para "Sitaq" no Babel
channel_id = '2'
manifestation_type_id = '2'
collect_id = '2'

# Discurso de teste
discurso = {}
discurso['id'] = '187.3.55.O 2 0 2'
discurso['autor'] = {'id':'Paulo Henriqu','nome':'Paulo Henriqu','tipo_autor':'Deputado','UF':'DF','partido':'PPPP','sexo':'M'}
discurso['integra'] = 'Já sofri mais. Agora estou colhendo alguns risos.    Boa tarde a todos.    Primeiro, quero cumprimentar a Mesa: a Gianna Sagazio, da CNIé uma alegria poder estar aqui neste momento de luta pela Ciência; o Prof. Luiz Davidovich, que muito nos honra com as defesas muito firmes, muito sólidas do nosso sistema de Ciência e Tecnologia; o Deputado e ex-Ministro Celso Pansera, um grande aliado em todas as nossas lutas; e a Profa. Helena Nader, a quem estendo um caloroso abraço pelos 10 anos de luta à frente da SBPCé com muito orgulho que contamos esse tempo todo com a sua brava e denodada luta à frente do nosso sistema.    O Parlamento é, sem dúvida, uma Casa que vem defendendo com força o nosso setor de Ciência, Tecnologia e inovação. Em 2015, com a Emenda Constitucional nº 85, tivemos ganhos bastante relevantes, como a institucionalização do Sistema Nacional de Ciência e Tecnologia, ainda pendente de institucionalização por meio de lei.    Em 2016, no ano seguinte, obtivemos a regulamentação da emenda constitucional, como marco legal, com a Lei nº 13.243. Infelizmente, houve oito vetos, ainda pendentes de reversão. Existem algumas iniciativas no Congresso para isso. Destacaria o Projeto de Lei do Senado nº 226, de 2016, de autoria do Senador Jorge Viana e com a relatoria do Senador Cristovam Buarque. Se tudo der certo, conseguiremos como vitória deste ano a reversão desses vetos.    A criação do marco legal é, sem dúvida, um avanço fantástico. Lutamos duramente pela sua regulamentação. Existem mais dificuldades internas do que externas para o alcance do que foi colocado, em termos de Constituição, em termos de marco legal. Mas seguimos. Seguimos também em outras frentes. Por exemplo, a retomada do Conselho Nacional de Ciência e Tecnologia, o CCT. Estamos cumprindo uma agenda consecutiva de debates, para a tomada de ações efetivas com base nesse fórum, que é o fórum máximo do nosso setor, presidido pelo Presidente da República.    Quero agradecer ao representante da Fundação de Apoio à Pesquisa do Distrito FederalFAP-DF por ter tratado da questão da Estratégia Nacional de Ciência, Tecnologia e InovaçãoENCTI. Ela foi debatida e aprovada no âmbito do CCT e tem um horizonte temporal de 2016 a 2022. Contamos com 12 áreas estratégicas e, neste momento, estamos trabalhando no desdobramento delas, em planos operacionais, em planos que vão detalhar de que modo vamos conseguir fazer, até 2022, as ações mais específicas e mais concretas para o nosso sistema.    Sem dúvida, as restrições orçamentárias nos apresentam grandes desafios, os cortes são realmente drásticos, mas seguimos fazendo todo o esforço possível para planejar o nosso sistema com base nessa ENCTI, que tem o horizonte temporal estendido para 2022.    Outra iniciativa, também de planejamento, da qual estamos nos aproximando, que tem horizonte temporal mais estendido, são os 17 Objetivos de Desenvolvimento Sustentável da ONU, que vão vigorar de 2016 a 2030, uma iniciativa à qual o mundo inteiro se associa e da qual o Ministério da Ciência e Tecnologia tem promovido grandes esforços para se aproximar.    No campo da regulação, o Jorge Mario Campagnolo, que está aqui, o Diretor da SETECSecretaria de Desenvolvimento Tecnológico e Inovação e Secretário Substituto, está à frente das discussões sobre a revisão da Lei do Bem. Além disso, temos a própria regulamentação da Lei 13.243 e estamos trabalhando firmemente também na revisão da Lei de Informática.    Vale destacar que esta semana tivemos um avanço relacionado às Fundações de Apoio, com a extensão do prazo de credenciamento de 2 anos para 5 anos. Esse era um gargalo muito grande no sistema. Esperamos tornar isso lei a partir da Medida Provisória nº 785.    Aqui nesta Casa há outra iniciativa muito relevante, que é a da criação dos fundos patrimoniais, um instrumento de alta relevância no exterior e que esperamos conseguir institucionalizar no País. Então, a criação dos fundos patrimoniais é uma ação também bastante relevante.    Em relação às ações mais concretas, em termos de editais, em termos de ações efetivas, o Marcelo Morales, do CNPq, representante do Mario Neto, já abordou várias delas. Sem dúvida, conseguimos ainda tocar algumas ações, apesar dos cortes, com muito esforço, com muita reorientação de iniciativas.    Eu destacaria também que vamos manter a Semana Nacional de Ciência e Tecnologia, a nossa principal ação de popularização, no Sistema Nacional de Ciência, Tecnologia e Inovação. É um grande evento, de dimensão nacional, realizado no mês de outubro.    Enfim, tratando mais especificamente dos cortes orçamentários, o desafio é enorme. Ontem pela manhã, no Senado Federal, foi colocado pelo nosso Secretário Executivo Elton Zacarias que esforços vêm sendo empreendidos pelo Ministro Gilberto Kassab junto ao Planejamento, junto à Fazenda, para recompor isso. Não é fácil a discussão, mas a expectativa é de que até setembro consigamos ter boas notícias nesse sentido. Tudo depende, obviamente, de novas receitas, de um novo respiro da economia, para conseguirmos fazer essa recomposição, na medida do possível.    O corte é drástico. Assim como concordo com o Secretário Elton, tenho que concordar com todas as falas feitas aqui, no mérito. Todas elas são realmente relevantes e direcionadas ao ponto correto da questão das dificuldades orçamentárias. Se não conseguirmos superar isso, se não conseguirmos superar a capacidade de financiamento, de investimento em Ciência, Tecnologia e inovação no País, sem dúvida alguma, não teremos nenhum futuro digno a apresentar aos nossos filhos, aos nossos netos; sem dúvida alguma, todo o nosso desenvolvimento econômico, social, ambiental vai estar comprometido por algumas gerações, por todos os motivos que já foram enumerados aqui.    Deputado Celso Pansera, mais uma vez agradeço a oportunidade de estar aqui. Parabenizo-o pela iniciativa. Sigamos em frente nesta luta.    Obrigado.   (Palmas.)'
discurso['original'] = 'Já sofri mais. Agora estou colhendo alguns risos.    Boa tarde a todos.    Primeiro, quero cumprimentar a Mesa: a Gianna Sagazio, da CNIé uma alegria poder estar aqui neste momento de luta pela Ciência; o Prof. Luiz Davidovich, que muito nos honra com as defesas muito firmes, muito sólidas do nosso sistema de Ciência e Tecnologia; o Deputado e ex-Ministro Celso Pansera, um grande aliado em todas as nossas lutas; e a Profa. Helena Nader, a quem estendo um caloroso abraço pelos 10 anos de luta à frente da SBPCé com muito orgulho que contamos esse tempo todo com a sua brava e denodada luta à frente do nosso sistema.    O Parlamento é, sem dúvida, uma Casa que vem defendendo com força o nosso setor de Ciência, Tecnologia e inovação. Em 2015, com a Emenda Constitucional nº 85, tivemos ganhos bastante relevantes, como a institucionalização do Sistema Nacional de Ciência e Tecnologia, ainda pendente de institucionalização por meio de lei.    Em 2016, no ano seguinte, obtivemos a regulamentação da emenda constitucional, como marco legal, com a Lei nº 13.243. Infelizmente, houve oito vetos, ainda pendentes de reversão. Existem algumas iniciativas no Congresso para isso. Destacaria o Projeto de Lei do Senado nº 226, de 2016, de autoria do Senador Jorge Viana e com a relatoria do Senador Cristovam Buarque. Se tudo der certo, conseguiremos como vitória deste ano a reversão desses vetos.    A criação do marco legal é, sem dúvida, um avanço fantástico. Lutamos duramente pela sua regulamentação. Existem mais dificuldades internas do que externas para o alcance do que foi colocado, em termos de Constituição, em termos de marco legal. Mas seguimos. Seguimos também em outras frentes. Por exemplo, a retomada do Conselho Nacional de Ciência e Tecnologia, o CCT. Estamos cumprindo uma agenda consecutiva de debates, para a tomada de ações efetivas com base nesse fórum, que é o fórum máximo do nosso setor, presidido pelo Presidente da República.    Quero agradecer ao representante da Fundação de Apoio à Pesquisa do Distrito FederalFAP-DF por ter tratado da questão da Estratégia Nacional de Ciência, Tecnologia e InovaçãoENCTI. Ela foi debatida e aprovada no âmbito do CCT e tem um horizonte temporal de 2016 a 2022. Contamos com 12 áreas estratégicas e, neste momento, estamos trabalhando no desdobramento delas, em planos operacionais, em planos que vão detalhar de que modo vamos conseguir fazer, até 2022, as ações mais específicas e mais concretas para o nosso sistema.    Sem dúvida, as restrições orçamentárias nos apresentam grandes desafios, os cortes são realmente drásticos, mas seguimos fazendo todo o esforço possível para planejar o nosso sistema com base nessa ENCTI, que tem o horizonte temporal estendido para 2022.    Outra iniciativa, também de planejamento, da qual estamos nos aproximando, que tem horizonte temporal mais estendido, são os 17 Objetivos de Desenvolvimento Sustentável da ONU, que vão vigorar de 2016 a 2030, uma iniciativa à qual o mundo inteiro se associa e da qual o Ministério da Ciência e Tecnologia tem promovido grandes esforços para se aproximar.    No campo da regulação, o Jorge Mario Campagnolo, que está aqui, o Diretor da SETECSecretaria de Desenvolvimento Tecnológico e Inovação e Secretário Substituto, está à frente das discussões sobre a revisão da Lei do Bem. Além disso, temos a própria regulamentação da Lei 13.243 e estamos trabalhando firmemente também na revisão da Lei de Informática.    Vale destacar que esta semana tivemos um avanço relacionado às Fundações de Apoio, com a extensão do prazo de credenciamento de 2 anos para 5 anos. Esse era um gargalo muito grande no sistema. Esperamos tornar isso lei a partir da Medida Provisória nº 785.    Aqui nesta Casa há outra iniciativa muito relevante, que é a da criação dos fundos patrimoniais, um instrumento de alta relevância no exterior e que esperamos conseguir institucionalizar no País. Então, a criação dos fundos patrimoniais é uma ação também bastante relevante.    Em relação às ações mais concretas, em termos de editais, em termos de ações efetivas, o Marcelo Morales, do CNPq, representante do Mario Neto, já abordou várias delas. Sem dúvida, conseguimos ainda tocar algumas ações, apesar dos cortes, com muito esforço, com muita reorientação de iniciativas.    Eu destacaria também que vamos manter a Semana Nacional de Ciência e Tecnologia, a nossa principal ação de popularização, no Sistema Nacional de Ciência, Tecnologia e Inovação. É um grande evento, de dimensão nacional, realizado no mês de outubro.    Enfim, tratando mais especificamente dos cortes orçamentários, o desafio é enorme. Ontem pela manhã, no Senado Federal, foi colocado pelo nosso Secretário Executivo Elton Zacarias que esforços vêm sendo empreendidos pelo Ministro Gilberto Kassab junto ao Planejamento, junto à Fazenda, para recompor isso. Não é fácil a discussão, mas a expectativa é de que até setembro consigamos ter boas notícias nesse sentido. Tudo depende, obviamente, de novas receitas, de um novo respiro da economia, para conseguirmos fazer essa recomposição, na medida do possível.    O corte é drástico. Assim como concordo com o Secretário Elton, tenho que concordar com todas as falas feitas aqui, no mérito. Todas elas são realmente relevantes e direcionadas ao ponto correto da questão das dificuldades orçamentárias. Se não conseguirmos superar isso, se não conseguirmos superar a capacidade de financiamento, de investimento em Ciência, Tecnologia e inovação no País, sem dúvida alguma, não teremos nenhum futuro digno a apresentar aos nossos filhos, aos nossos netos; sem dúvida alguma, todo o nosso desenvolvimento econômico, social, ambiental vai estar comprometido por algumas gerações, por todos os motivos que já foram enumerados aqui.    Deputado Celso Pansera, mais uma vez agradeço a oportunidade de estar aqui. Parabenizo-o pela iniciativa. Sigamos em frente nesta luta.    Obrigado.   (Palmas.)'
discurso['indexacao'] = 'Tecnologia, Ciência, CNI'
discurso['dtDiscurso'] = '2017-11-23T10:23:33Z'
discurso['dtAtualizacao'] = '2018-02-23T10:23:33Z'

# Busca o discurso no Babel
p = {'id_in_channel':discurso['id']}
discursos_babel = requests.get('http://babel.labhackercd.leg.br/api/v1/manifestations', params=p).json()['results']

# Inclui o discurso se ele não existir no Babel
if discursos_babel == []:
    
    # Pesquisa o autor do discurso no canal na base do Babel
    p = {'channel_id':channel_id, 'id_in_channel':discurso['autor']['id']}
    perfil = requests.get('http://babel.labhackercd.leg.br/api/v1/profiles', params=p).json()['results']

    # Cadastra o perfil se ele ainda não existir
    if perfil == []:

        # Busca o autor no Babel
        p = {'name':discurso['autor']['nome']}
        if 'tipo_autor' in discurso['autor']:
            p['author_type'] = discurso['autor']['tipo_autor']
        autor = requests.get('http://babel.labhackercd.leg.br/api/v1/authors', params=p).json()['results']

        # Cadastra o autor se ele ainda não existir
        if autor == []:
            p = {'name':discurso['autor']['nome'],
                 'author_type':discurso['autor']['tipo_autor']}
            if 'UF' in discurso['autor']:
                p['cep'] = CEP[discurso['autor']['UF']]
            if 'sexo' in discurso['autor']:
                p['gender'] = discurso['autor']['sexo']
            autor = requests.post('http://babel.labhackercd.leg.br/api/v1/authors', p).json()

        # Perfil encontrado no Babel
        else:
            autor = autor[0]

        print('Autor: ' + str(autor))

        # Obtém os atributos do perfil para cadastrá-lo
        p = {'channel_id':channel_id, 'id_in_channel':discurso['autor']['id'],'author_id':autor['id']}
        attrs = []
        if 'UF' in discurso['autor']:
            attrs.append({'field':'UF', 'value':discurso['autor']['UF']})
        if 'partido' in discurso['autor']:
            attrs.append({'field':'partido', 'value':discurso['autor']['partido']})
        p['attrs'] = attrs

        perfil = requests.post('http://babel.labhackercd.leg.br/api/v1/profiles', p).json()

    # Perfil encontrado no Babel
    else:
        perfil = perfil[0]

    print('Perfil: ' + str(perfil))

    # Obtém os atributos do discurso para cadastrá-lo
    p = {'manifestation_type_id':manifestation_type_id,
         'collect_id':collect_id,
         'profile_id':perfil['id'],
         'id_in_channel':discurso['id'],
         'content':discurso['integra'],
         'timestamp':discurso['dtAtualizacao']}
    attrs = []
    attrs.append({'field':'original', 'value':discurso['original']})
    attrs.append({'field':'dtDiscurso', 'value':discurso['dtDiscurso']})
    attrs.append({'field':'dtAtualizacao', 'value':discurso['dtAtualizacao']})
    p['attrs'] = attrs

    discurso_babel = requests.post('http://babel.labhackercd.leg.br/api/v1/manifestations', json=p).json()

    print(discurso_babel)

    
# Se o discurso já existir no Babel, verifica se a data de atualização é diferente
else:

    # Verifica, pela dtAtualizacao, se o discurso já está gravado no Babel com esta versão coletada
    nova_versao = True
    for discurso_babel in discursos_babel:
        if discurso_babel['attrs']. # PAREI!!!BUSCA O field 'dtAtualizacao' == discurso['dtAtualizacao']:
            nova_versão = False
    
    # Se for uma versão ainda inexistente, calcula o código da nova versão
    if nova_versao:
        max_version = 1
        if discurso_babel['version'] > max_version:
            max_version =discurso_babel['version']
            
        # Obtém os atributos do discurso para cadastrá-lo
        p = {'manifestation_type_id':manifestation_type_id,
             'collect_id':collect_id,
             'profile_id':perfil['id'],
             'id_in_channel':discurso['id'],
             'content':discurso['integra'],
             'timestamp':discurso['dtAtualizacao']}
        attrs = []
        attrs.append({'field':'original', 'value':discurso['original']})
        attrs.append({'field':'dtDiscurso', 'value':discurso['dtDiscurso']})
        attrs.append({'field':'dtAtualizacao', 'value':discurso['dtAtualizacao']})
        p['attrs'] = attrs

        # Grava o novo número de versão da manifestação/discurso
        p['version'] = str(max_version+1)

        discurso_babel = requests.post('http://babel.labhackercd.leg.br/api/v1/manifestations', json=p).json()
           
    
    print('Discurso existente!!!! ' + str(discurso_babel))


KeyError: 'dtAtualizacao'